### Part 1: Tournament preparation

#### 1. Teams vs Matches

Team definition

In [1]:
teams_eight = ['SCOTLAND','ENGLAND','ITALY','FRANCE', 'WALES', 'IRELAND', 'SPAIN', 'PORTUGAL']
teams_six = ['SCOTLAND','ENGLAND','ITALY','FRANCE', 'WALES', 'IRELAND']
teams_four = ['SCOTLAND','ENGLAND','ITALY','FRANCE']

Matches generation

In [2]:

def get_all_matches(teams):
    matches = []
    n = len(teams)
    for i in range(n):
        for j in range(n):
            team_1 = teams[i]
            team_2 = teams[j]
            if team_1 != team_2:
                matches.append((teams[i], teams[j]))

    return matches

#### 2. Constraint setup

a. Each round, all 6 teams of all matches must be different

In [3]:
def goal_test_condition_1(test_input):
    for triplet in test_input:
        seen = []
        for match in triplet:
            for team in match:
                if team in seen:
                    return False
                seen.append(team)
    return True

b. There can be no duplicate combination of teams considering all matches in a round

In [4]:
def goal_test_condition_2(test_input):
    seen = []
    for triplet in test_input:
        for match in triplet:
            match_reversed = (match[1], (match[0]))
            if (match in seen) | (match_reversed in seen):
                return False
            seen.append(match)
    return True

c. The last round must contain one match between FRANCE and ENGLAND, the order does not matter

In [5]:
def goal_test_condition_3(test_input):
    last_round = test_input[len(test_input)-1]
    match_FR_ENG = ('FRANCE', 'ENGLAND')
    match_FR_ENG_rev = ('ENGLAND', 'FRANCE')
    if ((match_FR_ENG in last_round ) | (match_FR_ENG_rev in last_round )):
        return True
    else:
        return False

d. No team plays against ITALY or FRANCE away twice in consecutive rounds

In [6]:
def goal_test_condition_4(test_input):
    opponents_current = []
    for triplet in test_input:
        opponents_new = []
        for match in triplet:
            if (match[0] in ('ITALY','FRANCE')):
                if (match[1] in opponents_current):
                    return False
                opponents_new.append(match[1])
        opponents_current = opponents_new
    return True

e. No two teams can play twice at home in consecutive rounds, or twice away in consecutive rounds

In [7]:
def goal_test_condition_5(test_input):
    home_current = []
    away_current = []
    for triplet in test_input:
        home_new = []
        away_new = []
        for match in triplet:
            if ((match[0] in home_current) | (match[1] in away_current)):
                return False
            home_new.append(match[0])
            away_new.append(match[1])
        home_current = home_new
        away_current = away_new
    return True

#### 3. Graph generation for each round

a. Check the duplicate combination in a round, regardless the order of matches

In [8]:
def check_unique_order(arr, new_item):
    contains_new_item = any(all(sub_item in tpl for sub_item in new_item) for tpl in arr)
    if contains_new_item:
        return False
    else:
        return True

b. Check the mirror match appear in the same round

In [9]:
def check_mirror(arr, new_match):
    contains_item = any(set(new_match).issubset(sublist) for sublist in arr)
    if contains_item:
        return False
    return True

c. Check duplicate team is a round, it means 1 team appear once time

In [10]:
def check_existed(arr, m):
    for a in arr:
        if m[0] in a or m[1] in a:
            return False
    return True

##### Graph generation for 1 round

In [11]:
def generate_graph(matches, current_round, rounds, teams):
    if len(current_round) == len(teams)/2:
        combined_matches = ()
        for tpl in current_round:
            combined_matches += (tpl,)
        if(check_unique_order(rounds, combined_matches)):
            rounds.append(combined_matches)
        return
    for i in range(len(matches)):
        match = matches[i]
        new_matches = []
        for m in matches[i:]:
            if check_mirror([match, new_matches], m) and check_existed([match, new_matches], m):
                new_matches.append(m)
        generate_graph(new_matches, current_round + [match], rounds, teams)
    return rounds

#### 4. Tournament generation

In [12]:
teams = teams_eight

Get all possible matches

In [13]:
matches = get_all_matches(teams)

Constraint enable?

In [14]:
constraint_enabled = True

#### a. Build tree for tournament

In [15]:
def build_tree(depth, match_round):
    tree = {triplet : match_round for triplet in match_round}
    counter = 1
    while counter < depth:
        print("Tree is growing")
        tree_grown = {key:tree for key in tree.keys()}
        tree = tree_grown
        counter += 1
    return tree

#### b. Result of tree

In [16]:

rounds = generate_graph(matches,[],[], teams)
graph = build_tree(len(teams)-1, rounds)

Tree is growing
Tree is growing
Tree is growing
Tree is growing
Tree is growing
Tree is growing


### Part 2: Searching solution

#### 1. DFS

In [17]:
def dfs(node, constraints_enabled, layer, path, graph, teams):
    if layer < len(teams) - 1:
        path[layer] = node
        if goal_test_condition_2(path[:layer+1]) == False:
            return None
        next_nodes = graph[node]
        if layer == len(teams)-2:
            next_nodes = graph
            if constraints_enabled:
                if goal_test_condition_4(path) and goal_test_condition_3(path):
                    return path
            else:
                return path
        for next_node in next_nodes.keys():
            result = dfs(next_node, constraints_enabled, layer + 1, path, next_nodes, teams)
            if result is not None:
                return result
    return None

def find_solution(graph, constraints_enabled, teams):
    path = [0 for _ in range(len(teams) - 1)]
    layer = 0
    for node in graph.keys():
        solution = dfs(node, constraints_enabled, layer, path, graph, teams)
        if solution is not None:
            return path
    return "No solution."


In [18]:
import time
counter = 0
mean = 0
while counter < 1:
  counter += 1
  t0 = time.time()
  result = find_solution(graph, constraint_enabled, teams)
  result
  t1 = time.time()
  total = t1-t0
  mean += total
print(mean/1)


KeyboardInterrupt: 

#### 2. BFS

In [ ]:
graph_bfs = graph.copy()

In [ ]:
def bfs(graph_bfs):
    layers = 0
    current_layer = [(graph_bfs, ['root'])]
    while current_layer:
        next_layer = []
        for node_dict, path in current_layer:
            for key, value in node_dict.items():
                if isinstance(value, dict):
                    if layers > 0:
                        if ((goal_test_condition_2(path[1:]+[key]) == True) and (goal_test_condition_4(path[1:]+[key]) == True)):
                            next_layer.append((value, path + [key]))
                    else:
                        next_layer.append((value, path + [key]))
                else:
                    for v in value:
                        if goal_test_condition_2(path[1:]+[v]) == True and goal_test_condition_4(path[1:]+[v]) == True:
                            if goal_test_condition_3(path[1:]+[v]):
                                return path[1:]+[v]
        current_layer = next_layer
        layers += 1

    return "No solution"

In [ ]:
def bfs_without_constraint(graph_bfs):
    layers = 0
    current_layer = [(graph_bfs, ['root'])]
    while current_layer:
        next_layer = []
        for node_dict, path in current_layer:
            for key, value in node_dict.items():
                if isinstance(value, dict):
                    if layers > 0:
                        if (goal_test_condition_2(path[1:]+[key]) == True):
                            next_layer.append((value, path + [key]))
                    else:
                        next_layer.append((value, path + [key]))
                else:
                    for v in value:
                        if goal_test_condition_2(path[1:]+[v]) == True:
                            return path[1:]+[v]
        current_layer = next_layer
        layers += 1

    return "No solution"

In [ ]:
import time

t0 = time.time()
result = bfs_without_constraint(graph_bfs)
result
t1 = time.time()

total = t1-t0
print(total)

### Part 3: Tournament found

In [ ]:
def createMirror(tournament):
    mirrored = tournament.copy()
    for r in tournament:
        round_i = []
        for m in r:
            round_i.append((m[1], m[0]))
        combined_matches = ()
        for tpl in round_i:
            combined_matches += (tpl,)
        mirrored.append(combined_matches)
    return mirrored

In [ ]:
tournament = createMirror(result)

In [ ]:
print("SOLUTION HAS BEEN FOUND, TOURNAMENT SCHEDULE IS AS FOLLOW:")
for i in range(len(tournament)):
    print(f"Round {i+1}: {tournament[i]}")

SOLUTION HAS BEEN FOUND, TOURNAMENT SCHEDULE IS AS FOLLOW:
Round 1: (('SCOTLAND', 'ENGLAND'), ('SCOTLAND', 'ITALY'))
Round 2: (('SCOTLAND', 'FRANCE'), ('ENGLAND', 'ITALY'))
Round 3: (('ENGLAND', 'FRANCE'), ('ITALY', 'FRANCE'))
Round 4: (('ENGLAND', 'SCOTLAND'), ('ITALY', 'SCOTLAND'))
Round 5: (('FRANCE', 'SCOTLAND'), ('ITALY', 'ENGLAND'))
Round 6: (('FRANCE', 'ENGLAND'), ('FRANCE', 'ITALY'))
